In [28]:
# Module 2 Challenge - Main Notebook for PyRamen Challenge

In [29]:
# Import the required libraries (ie pathlib, csv) (os for checking file presence) (sys to use exit function)
from pathlib import Path
import os  
import sys
import csv

In [30]:
# Initialise settings. Note: a convention of UPPERCASE has been adopted for constants

# NOTE: Following the README.md guidelines, an error "IOPub data rate exceeded error" was being produced due to the amount of error messages being printed (since there are 
# over 70,000 sales records, and the logic stated in the readme. An alternative approach has been offered here where the error is only produced once per menu item not sold.
### IMPLEMENTED_AS_PER_PYRAMEN_README = True     # Set to True: As per README.md logic, but produces this error "IOPub data rate exceeded error". Note output file is still generated correctly
IMPLEMENTED_AS_PER_PYRAMEN_README = False      # Set to False: Produces a warning message only once for each menu item with no sales        

# DEBUG: Shows debug messages when set to True
# --------------------------------------------
DEBUG = False

# INPUT_PATH: folder name where the input files reside
# ----------------------------------------------------
INPUT_PATH = "./Resources/"

# OUTPUT_PATH: folder name where the output report will be saved
# --------------------------------------------------------------
OUTPUT_PATH = "./"

# File characteristics: Filename, what column delimiters are used and whether there is an header exepected
# --------------------------------------------------------------------------------------------------------
MENU_DATA_FILENAME  = "menu_data.csv"      # Input filename menu data in csv format
MENU_DATA_DELIMITER = ','
MENU_DATA_HAS_HEADER = True
MENU_COL_IDX_ITEM = 0                      # Column index (starting at 0) in the csv file for the Description column
MENU_COL_IDX_PRICE = 3                     # Column index (starting at 0) in the csv file for the Price column
MENU_COL_IDX_COST = 4                      # Column index (starting at 0) in the csv file for the Cost column

SALES_DATA_FILENAME = "sales_data.csv"     # Input filename sales data in csv format
SALES_DATA_DELIMITER = ','
SALES_DATA_HAS_HEADER = True
SALES_COL_IDX_QTY = 3                      # Column index (starting at 0) in the csv file for the Quantity column
SALES_COL_IDX_MENU_ITEM = 4                # Column index (starting at 0) in the csv file for the Menu_Item column

REPORT_FILENAME = "report.txt"             # Output report text file where the report will be saved
REPORT_DELIMITER = ','
REPORT_HAS_HEADER = True

In [31]:
# =========================================
# Initialise local variables
# =========================================
# File handling variables
menu_data_abs_filename = ""           # Operating system independent absolute path and filename to the input menu file
sales_data_abs_filename = ""         # Operating system independent absolute path and filename to the input sales data file
report_abs_filename = ""              # Operating system independent absolute path and filename to the output report file

# For reporting results
output_text = ""                         # Buffer where the results are formatted into and used for both output to terminal and to the output report file

In [32]:
# Obtain a path to the input and output files which is compatible with the user's operating system
menu_data_abs_filename = os.path.abspath(INPUT_PATH + MENU_DATA_FILENAME)
sales_data_abs_filename = os.path.abspath(INPUT_PATH + SALES_DATA_FILENAME)
report_abs_filename = os.path.abspath(OUTPUT_PATH + REPORT_FILENAME)

if DEBUG:
    print(f"menu_data_abs_filename: {menu_data_abs_filename}")
    print(f"sales_data_abs_filename: {sales_data_abs_filename}")
    print(f"report_abs_filename: {report_abs_filename}")

In [33]:
# Check if the menu data file exists, if not show an error message and abort
if (not os.path.exists(menu_data_abs_filename)):
    print(f'Error: Input file "{menu_data_abs_filename}" does not exist')
    exit(1)      # Abort with error code

In [34]:
# Check if the sales data file exists, if not show an error message and abort
if (not os.path.exists(sales_data_abs_filename)):
    print(f'Error: Input file "{sales_data_abs_filename}" does not exist')
    exit(1)      # Abort with error code

In [35]:
# Read in the Menu Data file
# --------------------------

menu_list=[]      # Initialise menu list where the contents of the menu data file will be loaded

# Get a file object for the input file
with open( menu_data_abs_filename, 'r') as menu_data_file_obj:
    # Get a csv object from the input file object
    menu_data_csv_obj = csv.reader(menu_data_file_obj, delimiter=MENU_DATA_DELIMITER)

    if (MENU_DATA_HAS_HEADER):      # Skip header if present in the file
        next(menu_data_csv_obj)
        if DEBUG:
            print(f"Skipping header record in {MENU_DATA_FILENAME}")
    
    for menu_item in menu_data_csv_obj:
        menu_list.append( menu_item )      

In [36]:
# Read in the Sales Data file (only the Quantity and Menu Item data)
# ---------------------------

sales_list=[]     # Initialise sales list where the contents of the sales data file will be loaded
report_dict={}    # Initialise the report dictionary which will be used to output the report

# Get a file object for the input file
with open( sales_data_abs_filename, 'r') as sales_data_file_obj:
    # Get a csv object from the input file object
    sales_data_csv_obj = csv.reader(sales_data_file_obj, delimiter=SALES_DATA_DELIMITER)

    if (SALES_DATA_HAS_HEADER):     # Skip header if present in the file
        next(sales_data_csv_obj)
        if DEBUG:
            print(f"Skipping header record in {MENU_DATA_FILENAME}")
    
    # "For each row of the sales data, set the following columns of the sales data to their own variables: Quantity  Menu_Item" 
    for sales_item in sales_data_csv_obj:
        quantity = sales_item[SALES_COL_IDX_QTY]
        menu_item = sales_item[SALES_COL_IDX_MENU_ITEM]
        sales_list.append( [quantity, menu_item ] ) # Add only the Quantity and Menu Item data to the sales list being loaded
        
        if not (menu_item in report_dict):      # sales item is not in the report yet so add and initialise the counters and totals
            report_dict[menu_item] = { "01-count": 0, "02-revenue": 0, "03-cogs": 0, "04-profit": 0 }

if DEBUG:
    print( "Report Dict now loaded with: ", report_dict )

In [37]:
item_description = ""
item_price = 0
item_cost = 0

# Create a nested loop by looping through every record in menu.

for menu_item in menu_list:                              # For each row of the menu data, set the following columns of the menu data to their own variables:
    item_description = menu_item[MENU_COL_IDX_ITEM]      #   Item
    item_price = float(menu_item[MENU_COL_IDX_PRICE])    #   Price
    item_cost = float( menu_item[MENU_COL_IDX_COST])     #   Cost

    if not IMPLEMENTED_AS_PER_PYRAMEN_README:         # Alternate solution which only shows the error message once for items not sold (ie menu item is not in the sales list)
        if not item_description in sales_list:        # If there are no sales for this menu item
            print (f"{item_description} NO SALES!" )  #    Print the error only 1 time
                
    for sales_item in sales_list:
        if item_description == sales_item[1]:                 # If the sales_item in (the reduced) sales (list) is equal to the item in menu...
            qty_sold = int(sales_item[0])                     #   capture the quantity from the sales data and the price and cost from the menu data
            profit = item_price - item_cost                   #    to calculate the profit for each item.

            # Cumulatively add the values to the corresponding metrics in the report like so:
            report_dict[menu_item[0]]["01-count"] += qty_sold 
            report_dict[menu_item[0]]["02-revenue"] += item_price * qty_sold 
            report_dict[menu_item[0]]["03-cogs"] += item_cost * qty_sold 
            report_dict[menu_item[0]]["04-profit"] += profit * qty_sold
        else:
            if IMPLEMENTED_AS_PER_PYRAMEN_README:                                         # This solution is based on the README.md which causes an "IOPub data rate exceeded error"
                print (f"{sales_item[1]} does not equal {item_description}! NO MATCH!" )  # Print error for EVERY sales item that doesn't match (as per instructions in README)
 


edamame NO SALES!
house salad NO SALES!
cucumber sunomono salad NO SALES!
hiyashi wakame seaweed salad NO SALES!
agedashi tofu NO SALES!
hiyayakko tofu NO SALES!
pork floss bao NO SALES!
kakuni bao NO SALES!
edamame fried gyoza (vegan) NO SALES!
fried gyoza NO SALES!
takoyaki NO SALES!
rock shrimp tempura NO SALES!
soft-shell crab tempura NO SALES!
ebi katsu shrimp bao NO SALES!
nagomi shoyu NO SALES!
shio ramen NO SALES!
spicy miso ramen NO SALES!
vegetarian spicy miso NO SALES!
miso crab ramen NO SALES!
soft-shell miso crab ramen NO SALES!
tori paitan ramen NO SALES!
tonkotsu ramen NO SALES!
burnt garlic tonkotsu ramen NO SALES!
vegetarian curry + king trumpet mushroom ramen NO SALES!
truffle butter ramen NO SALES!
green tea NO SALES!
coke NO SALES!
black sesame ice cream NO SALES!
matcha ice cream NO SALES!
mango mochi ice cream NO SALES!
strawberry mochi ice cream NO SALES!
black sesame creme brulee NO SALES!


In [38]:
# Output the results to the terminal

with open(report_abs_filename, 'w') as report_file_handle:   # Open output file for writing
    for report_key, report_value in report_dict.items():     # loop through each entry in the report dictionary
        output_buffer = f"{report_key}: {report_value}\n"
        report_file_handle.write(output_buffer)
        print(output_buffer, end="")

# Tell the user where they can find the file with the report
print(f'\nA copy of the report has been saved to: "{report_abs_filename}" ')

spicy miso ramen: {'01-count': 9238, '02-revenue': 110856.0, '03-cogs': 46190.0, '04-profit': 64666.0}
tori paitan ramen: {'01-count': 9156, '02-revenue': 119028.0, '03-cogs': 54936.0, '04-profit': 64092.0}
truffle butter ramen: {'01-count': 8982, '02-revenue': 125748.0, '03-cogs': 62874.0, '04-profit': 62874.0}
tonkotsu ramen: {'01-count': 9288, '02-revenue': 120744.0, '03-cogs': 55728.0, '04-profit': 65016.0}
vegetarian spicy miso: {'01-count': 9216, '02-revenue': 110592.0, '03-cogs': 46080.0, '04-profit': 64512.0}
shio ramen: {'01-count': 9180, '02-revenue': 100980.0, '03-cogs': 45900.0, '04-profit': 55080.0}
miso crab ramen: {'01-count': 8890, '02-revenue': 106680.0, '03-cogs': 53340.0, '04-profit': 53340.0}
nagomi shoyu: {'01-count': 9132, '02-revenue': 100452.0, '03-cogs': 45660.0, '04-profit': 54792.0}
soft-shell miso crab ramen: {'01-count': 9130, '02-revenue': 127820.0, '03-cogs': 63910.0, '04-profit': 63910.0}
burnt garlic tonkotsu ramen: {'01-count': 9070, '02-revenue': 1269

# Pseudo Code

1. Read the Data
    1. Read in menu_data.csv and set its contents to a separate list object.
    1. Initialize variables: empty menu list, sales_list, report_dictionary objects
    1. Load menu_data.csv into a list (menu_list) skipping the header record
    1. Load sales_data.csv into a separate list (sales_list) skipping the header record 
1. Manipulate the Data
    1. Initialize an empty report dictionary to hold the future aggregated per-product results. The report dictionary will eventually contain the following metrics:
        1. 01-count: the total quantity for each ramen type
        1. 02-revenue: the total revenue for each ramen type
        1. 03-cogs: the total cost of goods sold for each ramen type
        1. 04-profit: the total profit for each ramen type
    1. Then, loops through every row in the sales list object:
        1. For each row of the sales data, set the following columns of the sales data to their own variables: Quantity, Menu_Item
        1. Perform a quick check if the sales_item is already included in the report. If not, initialize the key-value pairs for the particular sales_item in the report. Then, set the sales_item as a new key to the report dictionary and the values as a nested dictionary containing the following:
        ```{
        "01-count": 0,
        "02-revenue": 0,
        "03-cogs": 0,
        "04-profit": 0,
        }``` 

        1. Create a nested loop by looping through every record in menu.
            1. For each row of the menu data, set the following columns of the menu data to their own variables: Item, Price, Cost
            1. If the sales_item in sales is equal to the item in menu, capture the quantity from the sales data and the price and cost from the menu data to calculate the profit for each item.
                1. Cumulatively add the values to the corresponding metrics in the report like so:
                ```
                report[sales_item]["01-count"] += quantity
                report[sales_item]["02-revenue"] += price * quantity
                report[sales_item]["03-cogs"] += cost * quantity
                report[sales_item]["04-profit"] += profit * quantity
                ```
            1. Else print the message "{sales_item} does not equal {item}! NO MATCH!".


        1. Write out the contents of the report dictionary to a text file. The report should output each ramen type as the keys and 01-count, 02-revenue, 03-cogs, and 04-profit metrics as the values for every ramen type as shown:
```
spicy miso ramen {'01-count': 9238, '02-revenue': 110856.0, '03-cogs': 46190.0, '04-profit': 64666.0}
tori paitan ramen {'01-count': 9156, '02-revenue': 119028.0, '03-cogs': 54936.0, '04-profit': 64092.0}
truffle butter ramen {'01-count': 8982, '02-revenue': 125748.0, '03-cogs': 62874.0, '04-profit': 62874.0}
tonkotsu ramen {'01-count': 9288, '02-revenue': 120744.0, '03-cogs': 55728.0, '04-profit': 65016.0}
vegetarian spicy miso {'01-count': 9216, '02-revenue': 110592.0, '03-cogs': 46080.0, '04-profit': 64512.0}
shio ramen {'01-count': 9180, '02-revenue': 100980.0, '03-cogs': 45900.0, '04-profit': 55080.0}
miso crab ramen {'01-count': 8890, '02-revenue': 106680.0, '03-cogs': 53340.0, '04-profit': 53340.0}
nagomi shoyu {'01-count': 9132, '02-revenue': 100452.0, '03-cogs': 45660.0, '04-profit': 54792.0}
soft-shell miso crab ramen {'01-count': 9130, '02-revenue': 127820.0, '03-cogs': 63910.0, '04-profit': 63910.0}
burnt garlic tonkotsu ramen {'01-count': 9070, '02-revenue': 126980.0, '03-cogs': 54420.0, '04-profit': 72560.0}
vegetarian curry + king trumpet mushroom ramen {'01-count': 8824, '02-revenue': 114712.0, '03-cogs': 61768.0, '04-profit': 52944.0}
```